In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import seaborn as sns

r = requests.get('http://158.160.79.188:8000/GetAll')

data_raw = json.loads(r.content)['Frames']

#f = open('/GetAll.json')
#data_raw = json.load(f)["Frames"]


def flatten(rec):
    new_rec = {}
    for key, val in rec.items():
        if isinstance(val, dict):
            new_rec.update(flatten(rec[key]))
        else:
            new_rec[key] = val
    return new_rec

data = pd.DataFrame.from_records(map(flatten, data_raw))

data.head()

In [ ]:
data['TotalAcceleration'] = (data['AccelerationX'] ** 2 + data['AccelerationY'] ** 2 + data['AccelerationZ'] ** 2) ** 0.5
data['TotalAngleSpeed'] = (data['AngleSpeedX']**2 + data['AngleSpeedY']**2 + data['AngleSpeedZ']**2) ** 0.5
data['Time'] = data['Time'] - data['Time'][0]

In [ ]:
def addElevation(file):
    chunk_size = 100
    step = 5
    elev = []
    for first_elem in range(0, len(file), chunk_size * step):
        locations = '|'.join([f"{file['Latitude'][row + first_elem]},{file['Longitude'][row + first_elem]}" for row in range(0, min(chunk_size * step, len(file) - first_elem), step)])
        url = f"https://api.opentopodata.org/v1/mapzen?locations={locations}"
        response = requests.get(url)
        result = json.loads(response.text)
        for pos in range(0, min(chunk_size * step, len(file) - first_elem)):
          prev_pos = pos // step
          elev.append(result['results'][prev_pos]['elevation'])

    elevation_df = pd.DataFrame([{'Elevation': el} for el in elev])
    merged_df = pd.concat([file, elevation_df], axis=1)
    merged_df.to_csv('Data.csv', index=False)
    file = merged_df
    return file

In [ ]:
if 'elevation' not in data.columns:
    data = addElevation(data)

In [ ]:

data.plot('Time', 'Speed', subplots=True)
plt.xlim([0.5e6, 8e7])
#plt.ylim([0, 1])
plt.show()

In [ ]:
sns.pairplot(data)